In [2]:
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("The-ECG-Made-Easy-9th-Edition.pdf")
docs = loader.load()

In [3]:
print("Number of pages in the PDF:",len(docs))
docs[2].page_content


Number of pages in the PDF: 269


"Table of Contents\nCover image\nTitle Page\nCopyright\nHow to use this book\nPart 1: The ECG made very easy indeed: a beginner's guide\nPart 2: The basics: the fundamentals of ECG recording, reporting\nand interpretation\nPart 3: Making the most of the ECG: the clinical interpretation of\nindividual ECGs\nPart 4: Now test yourself\nQuick reminders\nFurther reading\nPreface\nPart 1 The ECG made very easy indeed: a beginner's\nguide\n"

In [6]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings

text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

/var/folders/8c/zybyw_md6p1c6nb6rz716x5h0000gn/T/ipykernel_24786/2762011161.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
/Users/sanjaibalajee/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
/Users/sanjaibalajee/.pyenv/versions/3.10.4/lib/python3.10/site-packag

In [7]:
for i in range(len(documents)):
    print()
    print(f"CHUNK : {i+1}")
    print(documents[i].page_content)


CHUNK : 1



CHUNK : 2
The ECG Made Easy
NINTH EDITION
John Hampton DM MA DPhil FRCP
FFPM FESC
Emeritus Professor of Cardiology, University of Nottingham, UK
Joanna Hampton MD MA BM BCh FRCP
Consultant Physician, Addenbrooke's Hospital, Cambridge, UK
EDINBURGH LONDON NEW YORK OXFORD
PHILADELPHIA ST LOUIS SYDNEY 2019


CHUNK : 3
Table of Contents
Cover image
Title Page
Copyright
How to use this book
Part 1: The ECG made very easy indeed: a beginner's guide
Part 2: The basics: the fundamentals of ECG recording, reporting
and interpretation
Part 3: Making the most of the ECG: the clinical interpretation of
individual ECGs
Part 4: Now test yourself
Quick reminders
Further reading
Preface
Part 1 The ECG made very easy indeed: a beginner's
guide


CHUNK : 4
1 The ECG made very easy indeed
What is an ECG? When do you need an ECG?

CHUNK : 5
How to record an ECG? How to interpret an ECG: the basics
Rhythms you must be able to recognize
Patterns you must be able to recognize
The normal ECG and

In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store 
vector = FAISS.from_documents(documents, embedder)

/var/folders/8c/zybyw_md6p1c6nb6rz716x5h0000gn/T/ipykernel_24786/2495835858.py:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()


In [13]:
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("Normal ECG patterns")

In [14]:
retrieved_docs

[Document(metadata={'source': 'The-ECG-Made-Easy-9th-Edition.pdf', 'file_path': 'The-ECG-Made-Easy-9th-Edition.pdf', 'page': 208, 'total_pages': 269, 'Author': 'J.Hampton', 'CreationDate': "D:20190323092533+00'00'", 'Creator': 'calibre (3.40.1) [https://calibre-ebook.com]', 'ModDate': "D:20190323142234+03'00'", 'Producer': 'calibre 3.40.1 [https://calibre-ebook.com]', 'Title': 'The ECG Made Easy E-Book'}, page_content='9\nECGs you must be able to\nrecognize\nECGs with clinical scenarios 164\nECG descriptions and interpretations 172\nYou should now be able to recognize the common ECG patterns, and\nthis final chapter contains twelve 12-lead ECGs for you to interpret. But do not forget two important things: first, an ECG comes from an\nindividual patient and must be interpreted with the patient in mind,\nand second, there is little point in recording and interpreting an ECG\nunless you are prepared to take some action based on your findings. This is a theme developed in the companion to 

In [15]:
from langchain_community.llms import Ollama
llm = Ollama(model="phi3")

In [16]:
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

prompt = """
1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.\n
3. Keep the answer crisp and limited to 3,4 sentences.

Context: {context}

Question: {question}

Helpful Answer:"""


QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None,
              )

qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True,
              )

/var/folders/8c/zybyw_md6p1c6nb6rz716x5h0000gn/T/ipykernel_24786/622636874.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
/var/folders/8c/zybyw_md6p1c6nb6rz716x5h0000gn/T/ipykernel_24786/622636874.py:31: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain/
  combine_documents_chain = StuffDocumentsChain(
/var/folders/8c/zybyw_md6p1c6nb6rz716x5h0000gn/T/ipykernel_24786/622636874.py:38: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/v0.2/docs/versions/migrating_chains/retrieval_qa/
  qa = RetrievalQA(


In [17]:
print(qa("What are some features of normal ECG patterns?"))

/var/folders/8c/zybyw_md6p1c6nb6rz716x5h0000gn/T/ipykernel_24786/2075715214.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(qa("What are some features of normal ECG patterns?"))




> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.

3. Keep the answer crisp and limited to 3,4 sentences.

Context: Context:
content:9
ECGs you must be able to
recognize
ECGs with clinical scenarios 164
ECG descriptions and interpretations 172
You should now be able to recognize the common ECG patterns, and
this final chapter contains twelve 12-lead ECGs for you to interpret. But do not forget two important things: first, an ECG comes from an
individual patient and must be interpreted with the patient in mind,
and second, there is little point in recording and interpreting an ECG
unless you are prepared to take some action based on your findings. This is a theme developed in the companion to this book, 150 ECG
Cases. The following ECGs (1–12) are in no particular 